In [1]:
from dotenv import load_dotenv
load_dotenv()

import os

import pandas as pd

# Inegi

In [2]:
def prepareInegi():
    inegi = pd.read_csv('inegi.csv')
    inegi.drop(columns=['nom_estab', 'per_ocu', 'fecha_alta', 'nom_processed', 'tipoUniEco', 'localidad'], inplace=True)
    return inegi

In [3]:
inegi = prepareInegi()
inegi.entidad.value_counts()

VERACRUZ DE IGNACIO DE LA LLAVE             48459
YUCATAN                                     17180
SAN LUIS POTOSÍ                             15655
TAMAULIPAS                                  15576
SINALOA                                     15321
SONORA                                      13198
QUERÉTARO                                   12448
TABASCO                                     11959
QUINTANA ROO                                11275
ZACATECAS                                    8474
TLAXCALA                                     8267
Name: entidad, dtype: int64

In [4]:
def getInegiByState(state):
    return inegi[inegi['entidad']==state]

In [5]:
sinegi = getInegiByState('TAMAULIPAS')
sinegi.head()

,entidad,municipio,latitud,longitud,tipo,Cocina Económica,Antojitos,Pizza,Hamburguesas,Hot Dogs,...,Yucateca,Tabasqueña,Arrachera,Pastor,Birria,Barbacoa,Carnero,Pozole,Enchiladas,Chilaquiles
79856,TAMAULIPAS,Matamoros,25.869956,-97.514709,"['burg', 'burgu', 'hamburgues']",0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
79857,TAMAULIPAS,Altamira,22.391528,-97.930704,[],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79858,TAMAULIPAS,Valle Hermoso,25.670395,-97.815718,[],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79859,TAMAULIPAS,Victoria,23.729669,-99.152546,[],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79860,TAMAULIPAS,Altamira,22.334950,-97.865727,['restaur'],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
sinegi['municipio'].value_counts()

Matamoros             2384
Tampico               2161
Reynosa               1972
Nuevo Laredo          1757
Victoria              1550
Ciudad Madero         1156
Altamira              1053
El Mante               667
Río Bravo              542
Valle Hermoso          380
Miguel Alemán          222
González               211
San Fernando           171
Tula                   154
Aldama                 134
Camargo                109
Gustavo Díaz Ordaz     108
Xicoténcatl            104
Soto la Marina          95
Jaumave                 87
Padilla                 85
Ocampo                  74
Hidalgo                 61
Mier                    48
Antiguo Morelos         37
Abasolo                 36
Llera                   36
Jiménez                 30
Nuevo Morelos           29
Gómez Farías            27
Guerrero                27
Güémez                  11
Villagrán               11
San Carlos              10
Burgos                   9
Miquihuana               7
Palmillas                5
B

In [7]:
def inegiFilterByMunicipio(municipio, df):
    return df.drop(df[df['municipio']!=municipio].index)#, inplace=True)

In [8]:
sinegi = inegiFilterByMunicipio('Tampico', sinegi)

In [9]:
sinegi.head()

,entidad,municipio,latitud,longitud,tipo,Cocina Económica,Antojitos,Pizza,Hamburguesas,Hot Dogs,...,Yucateca,Tabasqueña,Arrachera,Pastor,Birria,Barbacoa,Carnero,Pozole,Enchiladas,Chilaquiles
79866,TAMAULIPAS,Tampico,22.273236,-97.874833,[],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79867,TAMAULIPAS,Tampico,22.255264,-97.861039,['com'],1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79868,TAMAULIPAS,Tampico,22.262607,-97.859094,['com'],1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79874,TAMAULIPAS,Tampico,22.213607,-97.849838,[],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79885,TAMAULIPAS,Tampico,22.302357,-97.860012,[],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
foods = ['Cocina Económica', 'Antojitos', 'Pizza', 'Hamburguesas', 'Hot Dogs', 'Sushi', 'Tamales', 'Mariscos', 
         'Pescado', 'Tacos', 'Carne', 'Asada', 'Panuchos', 'Cochinita', 'Pollo', 'Desayunos', 'Tortas', 'Mondongo', 
         'Menudo', 'Memelas', 'Empanadas', 'Chicharrón', 'Gorditas', 'Costillas', 'Carnitas', 'Doraditas', 
         'Baguettes', 'Parrilla', 'Huaraches', 'Rosticería', 'China', 'Yucateca', 'Tabasqueña', 'Arrachera', 
         'Pastor', 'Birria', 'Barbacoa', 'Carnero', 'Pozole', 'Enchiladas', 'Chilaquiles']

In [11]:
def getInegiResults(df):
    results = {}

    for food in foods:
        results[food] = df[food].sum()

    return results

In [12]:
inegi_results = getInegiResults(sinegi)
inegi_results

{'Cocina Económica': 185,
 'Antojitos': 156,
 'Pizza': 31,
 'Hamburguesas': 31,
 'Hot Dogs': 23,
 'Sushi': 2,
 'Tamales': 39,
 'Mariscos': 45,
 'Pescado': 11,
 'Tacos': 344,
 'Carne': 0,
 'Asada': 69,
 'Panuchos': 0,
 'Cochinita': 9,
 'Pollo': 113,
 'Desayunos': 2,
 'Tortas': 252,
 'Mondongo': 1,
 'Menudo': 0,
 'Memelas': 0,
 'Empanadas': 9,
 'Chicharrón': 1,
 'Gorditas': 111,
 'Costillas': 1,
 'Carnitas': 38,
 'Doraditas': 0,
 'Baguettes': 5,
 'Parrilla': 11,
 'Huaraches': 1,
 'Rosticería': 5,
 'China': 0,
 'Yucateca': 3,
 'Tabasqueña': 0,
 'Arrachera': 2,
 'Pastor': 10,
 'Birria': 0,
 'Barbacoa': 26,
 'Carnero': 0,
 'Pozole': 0,
 'Enchiladas': 1,
 'Chilaquiles': 1}

In [13]:
def getInegiDfResult(inegi_results):
    return pd.DataFrame(inegi_results.items(), columns = ['Index', 'InegiCount']).set_index('Index')

In [14]:
df = getInegiDfResult(inegi_results)
df.head()

,InegiCount
Index,
Cocina Económica,185
Antojitos,156
Pizza,31
Hamburguesas,31
Hot Dogs,23


# User Location - GeoPy API

In [15]:
from geopy.geocoders import Nominatim
#user_input = input('Introduce lugar:')

def userLocationGeocoding(string):
    geolocator = Nominatim(user_agent="food-business-recommender")
    return geolocator.geocode(string)

In [16]:
location = userLocationGeocoding('unidad modelo tampico tamaulipas')
location.latitude

22.2853942

# Google Trends

### Revisaremos qué tantas búsquedas se hacen en Google sobre cada uno de los giros de comida, con el fin de conocer dónde está el interés del consumidor.
### Para esto utilizaremos la API de Google Trends a través de pytrends.

In [17]:
from pytrends.request import TrendReq

In [18]:
def getGoogleTrends(foods, df):
    pytrends = TrendReq(hl='es-MX', tz=360)
    
    resultados = []
    for comida in foods:
        print(comida)
        pytrends.build_payload([comida], cat=71, geo='MX', gprop='')
        resultados.append(pytrends.interest_by_region(resolution='REGION', inc_low_vol=True, inc_geo_code=False))
    return resultados

In [19]:
google_trends = getGoogleTrends(foods, sinegi)
google_trends

Cocina Económica
Antojitos
Pizza
Hamburguesas
Hot Dogs
Sushi
Tamales
Mariscos
Pescado
Tacos
Carne
Asada
Panuchos
Cochinita
Pollo
Desayunos
Tortas
Mondongo
Menudo
Memelas
Empanadas
Chicharrón
Gorditas
Costillas
Carnitas
Doraditas
Baguettes
Parrilla
Huaraches
Rosticería
China
Yucateca
Tabasqueña
Arrachera
Pastor
Birria
Barbacoa
Carnero
Pozole
Enchiladas
Chilaquiles


[                      Cocina Económica
 geoName                               
 Aguascalientes                       0
 Baja California                      0
 Baja California Sur                  0
 Campeche                             0
 Chiapas                              0
 Chihuahua                            0
 Ciudad de México                    27
 Coahuila de Zaragoza                 0
 Colima                               0
 Durango                              0
 Estado de México                    23
 Guanajuato                           0
 Guerrero                             0
 Hidalgo                              0
 Jalisco                              0
 Michoacán                            0
 Morelos                              0
 Nayarit                              0
 Nuevo León                           0
 Oaxaca                               0
 Puebla                               0
 Querétaro                            0
 Quintana Roo                         0


### Como los resultados nos los da por estados, filtramos y nos quedamos con las búsquedas en Yucatán.

In [20]:
def getTrends(google_trends, state):
    requests = google_trends[0].join(google_trends[1:]).T
    return requests['Yucatán'].sort_values(ascending=False)

In [21]:
trends =  getTrends(google_trends, 'Tampico')
trends

Cocina Económica    100
Parrilla            100
Pizza               100
Carnero             100
Yucateca            100
Doraditas           100
Baguettes           100
Panuchos             99
Pastor               94
Cochinita            89
Mondongo             80
Pollo                78
Desayunos            74
Chilaquiles          70
Carne                65
Tamales              57
Chicharrón           55
Pescado              54
China                54
Costillas            52
Hamburguesas         50
Arrachera            46
Enchiladas           42
Carnitas             40
Huaraches            36
Asada                36
Empanadas            36
Memelas              35
Barbacoa             33
Antojitos            33
Mariscos             29
Sushi                29
Hot Dogs             27
Pozole               25
Tortas               21
Tacos                21
Birria               17
Gorditas             16
Tabasqueña           12
Menudo               10
Rosticería            0
Name: Yucatán, d

In [22]:
df['GoogleTrend'] = trends
df

,InegiCount,GoogleTrend
Index,,
Cocina Económica,185,100
Antojitos,156,33
Pizza,31,100
Hamburguesas,31,50
Hot Dogs,23,27
Sushi,2,29
Tamales,39,57
Mariscos,45,29
Pescado,11,54


## Interest over time

In [25]:
def getGoogleIOT(foods):
    pytrends = TrendReq(hl='es-MX', tz=360)
    iot = {}
    for comida in foods:
        print(comida)
        pytrends.build_payload([comida], cat=71, geo='MX', gprop='')
        iot[comida] = pytrends.interest_over_time()
        
    return iot

In [26]:
iot = getGoogleIOT(foods)
iot

Cocina Económica
Antojitos
Pizza
Hamburguesas
Hot Dogs
Sushi
Tamales
Mariscos
Pescado
Tacos
Carne
Asada
Panuchos
Cochinita
Pollo
Desayunos
Tortas
Mondongo
Menudo
Memelas
Empanadas
Chicharrón
Gorditas
Costillas
Carnitas
Doraditas
Baguettes
Parrilla
Huaraches
Rosticería
China
Yucateca
Tabasqueña
Arrachera
Pastor
Birria
Barbacoa
Carnero
Pozole
Enchiladas
Chilaquiles


{'Cocina Económica':             Cocina Económica isPartial
 date                                  
 2015-03-08                 0     False
 2015-03-15                 0     False
 2015-03-22                 0     False
 2015-03-29                 0     False
 2015-04-05                 0     False
 ...                      ...       ...
 2020-02-02                 0     False
 2020-02-09                52     False
 2020-02-16                26     False
 2020-02-23                 0     False
 2020-03-01                 0      True
 
 [261 rows x 2 columns], 'Antojitos':             Antojitos isPartial
 date                           
 2015-03-08         24     False
 2015-03-15         14     False
 2015-03-22         14     False
 2015-03-29         15     False
 2015-04-05         21     False
 ...               ...       ...
 2020-02-02         39     False
 2020-02-09         25     False
 2020-02-16         28     False
 2020-02-23         31     False
 2020-03-01         58   

In [ ]:
import matplotlib.pyplot as plt

x = [i+1 for i in range(len(iot['Desayunos']['Desayunos']))]

#for food in foods:
plt.plot(x, iot['Desayunos']['Desayunos'].values);

In [ ]:
iot['Gorditas']

In [27]:
import numpy as np
from sklearn.linear_model import LinearRegression

def getTrendSlope(foods, iot):
    x = [i+1 for i in range(len(iot['Desayunos']['Desayunos']))]
    linreg = LinearRegression()

    slope = {}
    for food in foods:
        linreg.fit(np.array(x).reshape(-1,1), iot[food][food].values)
        slope[food] = linreg.intercept_

    return slope

In [28]:
slope = getTrendSlope(foods, iot)
slope

{'Cocina Económica': 7.047421161214264,
 'Antojitos': 18.666106690244618,
 'Pizza': 40.0480695549661,
 'Hamburguesas': 10.274506336575302,
 'Hot Dogs': 42.25101679929266,
 'Sushi': 44.36206896551724,
 'Tamales': 13.383554376657823,
 'Mariscos': 20.15923961096375,
 'Pescado': 30.019039198349546,
 'Tacos': 37.825228411435305,
 'Carne': 40.17132331270262,
 'Asada': 10.858325965222516,
 'Panuchos': 26.046654877689356,
 'Cochinita': 40.51889183613322,
 'Pollo': 55.90279988211023,
 'Desayunos': 47.7621868552903,
 'Tortas': 44.565104627173596,
 'Mondongo': 12.989183613321543,
 'Menudo': 14.262157382847036,
 'Memelas': 24.903330386089003,
 'Empanadas': 20.801444149720012,
 'Chicharrón': 3.0745947539050995,
 'Gorditas': 45.71765399351606,
 'Costillas': 16.09746536987916,
 'Carnitas': 29.142145593869728,
 'Doraditas': 38.435307987032125,
 'Baguettes': 23.469024462127912,
 'Parrilla': 31.297023283230182,
 'Huaraches': 31.455585027998826,
 'Rosticería': 2.787326849395815,
 'China': 21.448099027409

In [29]:
def addSlopeToDf(df):
    df['slope'] = ''
    for key in slope:
        df['slope'].loc[key] = slope[key]
    df.slope = df.slope.astype('float64')
    
    return df

In [30]:
df = addSlopeToDf(df)
df.dtypes

/home/linuxbrew/.linuxbrew/opt/python/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


InegiCount       int64
GoogleTrend      int64
slope          float64
dtype: object

### Con esto, tenemos parametrizado el interés en el estado por cada uno de los giros de comida.
### Ahora, necesitamos revisar el grado de satisfacción de los usuarios por la oferta existente.

# Yelp

### Utilizando la API de Yelp, hacemos búsquedas nuevamente sobre los diferentes giros de comida para encontrar negocios y sus calificaciónes en la plataforma.

In [31]:
import requests
from bs4 import BeautifulSoup as bs
import json

In [32]:
def searchYelp(foods, location):
    api_key = os.getenv("YELP_API_KEY")
    endpoint = 'https://api.yelp.com/v3/businesses/search?'
    
    yelp_search = []
    for comida in foods:
        term = 'term={}&'.format(comida)
        print(term)
        latitude = 'latitude='+str(location.latitude)+'&'
        longitude = 'longitude='+str(location.longitude)+'&'
        locale = 'locale=es_MX&'
        radius = 'radius=3000&'
        limit = 'limit=50' 
        res = requests.get(endpoint+term+latitude+longitude+locale+radius+limit, headers={'Authorization':'Bearer '+api_key})
        soup = bs(res.content)
        yelp_json = json.loads(soup.findAll('p')[0].text)
        ydf = pd.DataFrame(yelp_json['businesses'])
        ydf['tipo'] = comida
        yelp_search.append(ydf)

    return yelp_search

In [33]:
yelp_search = searchYelp(foods, location)
yelp_search

term=Cocina Económica&
term=Antojitos&
term=Pizza&
term=Hamburguesas&
term=Hot Dogs&
term=Sushi&
term=Tamales&
term=Mariscos&
term=Pescado&
term=Tacos&
term=Carne&
term=Asada&
term=Panuchos&
term=Cochinita&
term=Pollo&
term=Desayunos&
term=Tortas&
term=Mondongo&
term=Menudo&
term=Memelas&
term=Empanadas&
term=Chicharrón&
term=Gorditas&
term=Costillas&
term=Carnitas&
term=Doraditas&
term=Baguettes&
term=Parrilla&
term=Huaraches&
term=Rosticería&
term=China&
term=Yucateca&
term=Tabasqueña&
term=Arrachera&
term=Pastor&
term=Birria&
term=Barbacoa&
term=Carnero&
term=Pozole&
term=Enchiladas&
term=Chilaquiles&


[Empty DataFrame
 Columns: [tipo]
 Index: [], Empty DataFrame
 Columns: [tipo]
 Index: [],                        id                    alias              name  \
 0  77jt2C-RMmBVn3d4YO3Ujg  chuck-e-cheeses-tampico  Chuck e Cheese's   
 
                                            image_url  is_closed  \
 0  https://s3-media3.fl.yelpcdn.com/bphoto/GLCEzI...      False   
 
                                                  url  review_count  \
 0  https://www.yelp.com.mx/biz/chuck-e-cheeses-ta...             1   
 
                                           categories  rating  \
 0  [{'alias': 'arcades', 'title': 'Sala de juegos...     4.0   
 
                                        coordinates transactions  \
 0  {'latitude': 22.27214, 'longitude': -97.873009}           []   
 
                                             location          phone  \
 0  {'address1': 'Av. Miguel Hidalgo S/N', 'addres...  +528338003030   
 
      display_phone     distance   tipo  
 0  01 833 800 3030  1

In [34]:
def getYelpDf(yelp_search):
    lugares = pd.concat(yelp_search, axis=0, sort=False)
    lugares.reset_index(inplace=True)

    ratings = dict(lugares.tipo.value_counts())
    for comida in foods:
        ratings[comida] = lugares[lugares.tipo == comida]['rating'].mean()

    yf = pd.concat(yelp_search, sort=False)
    yf.drop(columns = ['alias', 'display_phone', 'location', 'id', 'image_url', 'is_closed', 'phone', 'transactions', 'url'], inplace=True)
    yf.reset_index(inplace=True)
    return yf

In [36]:
yf = getYelpDf(yelp_search)
yf

,index,tipo,name,review_count,categories,rating,coordinates,distance,price
0,0,Pizza,Chuck e Cheese's,1.0,"[{'alias': 'arcades', 'title': 'Sala de juegos...",4.0,"{'latitude': 22.27214, 'longitude': -97.873009}",1979.691511,NaN
1,0,Hamburguesas,Marusha's,1.0,"[{'alias': 'burgers', 'title': 'Hamburguesas'}...",4.0,"{'latitude': 22.2638902126154, 'longitude': -9...",3413.126260,$
2,1,Hamburguesas,McDonalds,1.0,"[{'alias': 'burgers', 'title': 'Hamburguesas'}...",4.0,"{'latitude': 22.2756886931187, 'longitude': -9...",1663.131592,$$
3,2,Hamburguesas,7 Leguas,1.0,"[{'alias': 'tacos', 'title': 'Taquerías'}, {'a...",3.0,"{'latitude': 22.273538, 'longitude': -97.87504...",1724.937372,NaN
4,3,Hamburguesas,Carl's Jr,1.0,"[{'alias': 'burgers', 'title': 'Hamburguesas'}]",3.0,"{'latitude': 22.2657831010059, 'longitude': -9...",3433.658618,NaN
5,4,Hamburguesas,Applebee's Grill + Bar,1.0,"[{'alias': 'tradamerican', 'title': 'Cocina no...",1.0,"{'latitude': 22.272441, 'longitude': -97.875505}",1791.276534,$$
6,0,Sushi,Toyama Sushi Concept,1.0,"[{'alias': 'sushi', 'title': 'Sushi'}, {'alias...",4.0,"{'latitude': 22.2742, 'longitude': -97.86816}",1748.815983,NaN
7,1,Sushi,Kurai Sushi Bar,4.0,"[{'alias': 'sushi', 'title': 'Sushi'}]",4.0,"{'latitude': 22.257026, 'longitude': -97.874816}",3352.686655,$$
8,0,Mariscos,El Dorado,1.0,"[{'alias': 'buffets', 'title': 'Buffets'}, {'a...",4.0,"{'latitude': 22.3030333941498, 'longitude': -9...",2000.494185,NaN
9,0,Tacos,Tacos Ana,1.0,"[{'alias': 'streetvendors', 'title': 'Puestos ...",3.0,"{'latitude': 22.27917495095, 'longitude': -97....",793.579864,NaN


## Adding the mean rating per food to the main dataframe

In [37]:
df['YelpRating'] = yf.groupby('tipo').mean().rating
df

,InegiCount,GoogleTrend,slope,YelpRating
Index,,,,
Cocina Económica,185,100,7.047421,NaN
Antojitos,156,33,18.666107,NaN
Pizza,31,100,40.048070,4.000
Hamburguesas,31,50,10.274506,3.000
Hot Dogs,23,27,42.251017,NaN
Sushi,2,29,44.362069,4.000
Tamales,39,57,13.383554,NaN
Mariscos,45,29,20.159240,4.000
Pescado,11,54,30.019039,NaN


# Score

In [38]:
def calculateScore(df):
    #df['score'] = df.GoogleTrend/100 - df.YelpRating/5 - df.InegiCount/df.InegiCount.max()
    #df['score'] = 0.05*df.GoogleTrend/df.YelpRating - df.InegiCount/df.InegiCount.max() + 0.5*df.slope/df.slope.max()
    df['score'] = df.GoogleTrend/100 + df.slope/df.slope.max() - df.YelpRating/5 - df.InegiCount/df.InegiCount.max()
    return df.sort_values(by='score', ascending=False).head()

In [39]:
df = calculateScore(df)
df

,InegiCount,GoogleTrend,slope,YelpRating,score
Index,,,,,
Pizza,31,100,40.048070,4.000,0.770657
Parrilla,11,100,31.297023,4.000,0.684409
Carne,0,65,40.171323,3.375,0.637807
Sushi,2,29,44.362069,4.000,0.216138
Hamburguesas,31,50,10.274506,3.000,-0.020592


## Yelp Locations Dataframe

In [40]:
from pandas.io.json import json_normalize

In [41]:
def flatLatLong(yf):
    yf[['latitude', 'longitude']] = json_normalize(yf.coordinates)
    return yf

In [42]:
yf = flatLatLong(yf)
yf

,index,tipo,name,review_count,categories,rating,coordinates,distance,price,latitude,longitude
0,0,Pizza,Chuck e Cheese's,1.0,"[{'alias': 'arcades', 'title': 'Sala de juegos...",4.0,"{'latitude': 22.27214, 'longitude': -97.873009}",1979.691511,NaN,22.272140,-97.873009
1,0,Hamburguesas,Marusha's,1.0,"[{'alias': 'burgers', 'title': 'Hamburguesas'}...",4.0,"{'latitude': 22.2638902126154, 'longitude': -9...",3413.126260,$,22.263890,-97.862185
2,1,Hamburguesas,McDonalds,1.0,"[{'alias': 'burgers', 'title': 'Hamburguesas'}...",4.0,"{'latitude': 22.2756886931187, 'longitude': -9...",1663.131592,$$,22.275689,-97.873556
3,2,Hamburguesas,7 Leguas,1.0,"[{'alias': 'tacos', 'title': 'Taquerías'}, {'a...",3.0,"{'latitude': 22.273538, 'longitude': -97.87504...",1724.937372,NaN,22.273538,-97.875044
4,3,Hamburguesas,Carl's Jr,1.0,"[{'alias': 'burgers', 'title': 'Hamburguesas'}]",3.0,"{'latitude': 22.2657831010059, 'longitude': -9...",3433.658618,NaN,22.265783,-97.860078
5,4,Hamburguesas,Applebee's Grill + Bar,1.0,"[{'alias': 'tradamerican', 'title': 'Cocina no...",1.0,"{'latitude': 22.272441, 'longitude': -97.875505}",1791.276534,$$,22.272441,-97.875505
6,0,Sushi,Toyama Sushi Concept,1.0,"[{'alias': 'sushi', 'title': 'Sushi'}, {'alias...",4.0,"{'latitude': 22.2742, 'longitude': -97.86816}",1748.815983,NaN,22.274200,-97.868160
7,1,Sushi,Kurai Sushi Bar,4.0,"[{'alias': 'sushi', 'title': 'Sushi'}]",4.0,"{'latitude': 22.257026, 'longitude': -97.874816}",3352.686655,$$,22.257026,-97.874816
8,0,Mariscos,El Dorado,1.0,"[{'alias': 'buffets', 'title': 'Buffets'}, {'a...",4.0,"{'latitude': 22.3030333941498, 'longitude': -9...",2000.494185,NaN,22.303033,-97.882029
9,0,Tacos,Tacos Ana,1.0,"[{'alias': 'streetvendors', 'title': 'Puestos ...",3.0,"{'latitude': 22.27917495095, 'longitude': -97....",793.579864,NaN,22.279175,-97.882072


In [43]:
def getTopYfLocs(df, yf):
    top_types = df.sort_values(by='score', ascending=False).head().index

    top_yf_locs = {}
    for top in top_types:
        top_yf_locs[top] = yf[yf.tipo==top][['latitude', 'longitude']].head().values

    return top_yf_locs

In [44]:
top_yf_locs = getTopYfLocs(df, yf)
top_yf_locs

{'Pizza': array([[ 22.27214 , -97.873009]]),
 'Parrilla': array([[ 22.26262924, -97.85934569],
        [ 22.26483217, -97.87442158]]),
 'Carne': array([[ 22.26262924, -97.85934569],
        [ 22.27917495, -97.88207171],
        [ 22.27568869, -97.87355641],
        [ 22.30823636, -97.88094681]]),
 'Sushi': array([[ 22.2742  , -97.86816 ],
        [ 22.257026, -97.874816]]),
 'Hamburguesas': array([[ 22.26389021, -97.86218522],
        [ 22.27568869, -97.87355641],
        [ 22.273538  , -97.87504412],
        [ 22.2657831 , -97.86007848],
        [ 22.272441  , -97.875505  ]])}

In [45]:
yf

,index,tipo,name,review_count,categories,rating,coordinates,distance,price,latitude,longitude
0,0,Pizza,Chuck e Cheese's,1.0,"[{'alias': 'arcades', 'title': 'Sala de juegos...",4.0,"{'latitude': 22.27214, 'longitude': -97.873009}",1979.691511,NaN,22.272140,-97.873009
1,0,Hamburguesas,Marusha's,1.0,"[{'alias': 'burgers', 'title': 'Hamburguesas'}...",4.0,"{'latitude': 22.2638902126154, 'longitude': -9...",3413.126260,$,22.263890,-97.862185
2,1,Hamburguesas,McDonalds,1.0,"[{'alias': 'burgers', 'title': 'Hamburguesas'}...",4.0,"{'latitude': 22.2756886931187, 'longitude': -9...",1663.131592,$$,22.275689,-97.873556
3,2,Hamburguesas,7 Leguas,1.0,"[{'alias': 'tacos', 'title': 'Taquerías'}, {'a...",3.0,"{'latitude': 22.273538, 'longitude': -97.87504...",1724.937372,NaN,22.273538,-97.875044
4,3,Hamburguesas,Carl's Jr,1.0,"[{'alias': 'burgers', 'title': 'Hamburguesas'}]",3.0,"{'latitude': 22.2657831010059, 'longitude': -9...",3433.658618,NaN,22.265783,-97.860078
5,4,Hamburguesas,Applebee's Grill + Bar,1.0,"[{'alias': 'tradamerican', 'title': 'Cocina no...",1.0,"{'latitude': 22.272441, 'longitude': -97.875505}",1791.276534,$$,22.272441,-97.875505
6,0,Sushi,Toyama Sushi Concept,1.0,"[{'alias': 'sushi', 'title': 'Sushi'}, {'alias...",4.0,"{'latitude': 22.2742, 'longitude': -97.86816}",1748.815983,NaN,22.274200,-97.868160
7,1,Sushi,Kurai Sushi Bar,4.0,"[{'alias': 'sushi', 'title': 'Sushi'}]",4.0,"{'latitude': 22.257026, 'longitude': -97.874816}",3352.686655,$$,22.257026,-97.874816
8,0,Mariscos,El Dorado,1.0,"[{'alias': 'buffets', 'title': 'Buffets'}, {'a...",4.0,"{'latitude': 22.3030333941498, 'longitude': -9...",2000.494185,NaN,22.303033,-97.882029
9,0,Tacos,Tacos Ana,1.0,"[{'alias': 'streetvendors', 'title': 'Puestos ...",3.0,"{'latitude': 22.27917495095, 'longitude': -97....",793.579864,NaN,22.279175,-97.882072


In [46]:
def getTopYf(yf):
    top_yf = pd.DataFrame(columns = yf.columns)
    top_types = df.sort_values(by='score', ascending=False).head().index
    
    for top in top_types:
        top_yf = pd.concat([top_yf, yf[yf.tipo==top].sort_values(by='rating', ascending=False).head(10)])

    return top_yf

In [47]:
top_yf = getTopYf(yf)
top_yf

,index,tipo,name,review_count,categories,rating,coordinates,distance,price,latitude,longitude
0,0,Pizza,Chuck e Cheese's,1.0,"[{'alias': 'arcades', 'title': 'Sala de juegos...",4.0,"{'latitude': 22.27214, 'longitude': -97.873009}",1979.691511,NaN,22.272140,-97.873009
19,0,Parrilla,El Asador,10.0,"[{'alias': 'steak', 'title': 'Parrilla'}]",4.5,"{'latitude': 22.2626292435619, 'longitude': -9...",3721.330388,$$,22.262629,-97.859346
20,1,Parrilla,360 Grill And Bar,2.0,"[{'alias': 'bars', 'title': 'Bares'}, {'alias'...",3.5,"{'latitude': 22.2648321745431, 'longitude': -9...",2571.312031,$$,22.264832,-97.874422
13,0,Carne,El Asador,10.0,"[{'alias': 'steak', 'title': 'Parrilla'}]",4.5,"{'latitude': 22.2626292435619, 'longitude': -9...",3721.330388,$$,22.262629,-97.859346
15,2,Carne,McDonalds,1.0,"[{'alias': 'burgers', 'title': 'Hamburguesas'}...",4.0,"{'latitude': 22.2756886931187, 'longitude': -9...",1663.131592,$$,22.275689,-97.873556
14,1,Carne,Tacos Ana,1.0,"[{'alias': 'streetvendors', 'title': 'Puestos ...",3.0,"{'latitude': 22.27917495095, 'longitude': -97....",793.579864,NaN,22.279175,-97.882072
16,3,Carne,Sarape Oriental,1.0,"[{'alias': 'tacos', 'title': 'Taquerías'}]",2.0,"{'latitude': 22.3082363584824, 'longitude': -9...",2589.637945,NaN,22.308236,-97.880947
6,0,Sushi,Toyama Sushi Concept,1.0,"[{'alias': 'sushi', 'title': 'Sushi'}, {'alias...",4.0,"{'latitude': 22.2742, 'longitude': -97.86816}",1748.815983,NaN,22.274200,-97.868160
7,1,Sushi,Kurai Sushi Bar,4.0,"[{'alias': 'sushi', 'title': 'Sushi'}]",4.0,"{'latitude': 22.257026, 'longitude': -97.874816}",3352.686655,$$,22.257026,-97.874816
1,0,Hamburguesas,Marusha's,1.0,"[{'alias': 'burgers', 'title': 'Hamburguesas'}...",4.0,"{'latitude': 22.2638902126154, 'longitude': -9...",3413.126260,$,22.263890,-97.862185


In [48]:
top_yf.tipo.unique()

array(['Pizza', 'Parrilla', 'Carne', 'Sushi', 'Hamburguesas'],
      dtype=object)

# Foursquare

In [52]:
import foursquare

In [53]:
def getFsResults(top_yf):
    client_id = os.getenv("FS_CLIENT_ID")
    client_secret = os.getenv("FS_CLIENT_SECRET")
    client = foursquare.Foursquare(client_id=client_id, client_secret=client_secret)

    fs_results = {}
    for comida in top_yf.tipo.unique():#top_yf_locs.keys():
        print(comida)
        fs_results[comida] = client.venues.search(params={'query': comida, 'intent':'checkin', 'll': str(location.latitude)+', '+str(location.longitude), 'radius':3000, 'limit':10})

    return fs_results

In [54]:
fs_results = getFsResults(top_yf)
fs_results

Pizza
Parrilla
Carne
Sushi
Hamburguesas


{'Pizza': {'venues': [{'id': '5911187795d986095889a334',
    'name': 'Nosstra Pizza',
    'location': {'lat': 22.280081,
     'lng': -97.886614,
     'labeledLatLngs': [{'label': 'display',
       'lat': 22.280081,
       'lng': -97.886614}],
     'distance': 596,
     'postalCode': '89366',
     'cc': 'MX',
     'city': 'Tampico',
     'state': 'Tamaulipas',
     'country': 'México',
     'formattedAddress': ['89366 Tampico, Tamaulipas', 'México']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1583339081',
    'hasPerk': False},
   {'id': '4cef1467ed62721e61a35efd',
    'name': "Domino's Pizza",
    'location': {'address': 'Av Torres',
     'crossStreet': 'Tampico Mante',
     'lat': 22.31783023805583,
     'lng': -97.87794550550835,
  

In [55]:
def getFsLocs(fs_results):
    fs_locs = {}
    for result in fs_results:
        result_locs = []
        for i in range(len(fs_results[result]['venues'])):
            loc = {}
            loc[fs_results[result]['venues'][i]['name']] = []
            loc[fs_results[result]['venues'][i]['name']].append(fs_results[result]['venues'][i]['location']['lat'])
            loc[fs_results[result]['venues'][i]['name']].append(fs_results[result]['venues'][i]['location']['lng'])
            result_locs.append(loc)
        fs_locs[result] = result_locs

    return fs_locs

In [56]:
fs_locs = getFsLocs(fs_results)
fs_locs

{'Pizza': [{'Nosstra Pizza': [22.280081, -97.886614]},
  {"Domino's Pizza": [22.31783023805583, -97.87794550550835]},
  {'Tony Fran Pizza': [22.289217, -97.887514]},
  {"Vic's Pizza": [22.298737, -97.876295]},
  {'Pizza Hut': [22.262692477503265, -97.87394309471969]},
  {'Pizza Hut': [22.262543300232213, -97.87530704784369]},
  {"Little Caesar's Pizza": [22.27103218425978, -97.87444895124108]},
  {'Conni Pizza avenida hidalgo': [22.27343380767052, -97.86952143577572]},
  {'Peter Piper Pizza': [22.27546584670382, -97.87246012713402]},
  {"Domino's Pizza": [22.273370317585172, -97.87331000050186]}],
 'Parrilla': [{'restaurante  mi parrilla': [22.29838752746582,
    -97.89164733886719]},
  {'Bariloche! Parrilla Argentina': [22.275188143494294, -97.8711476390204]},
  {'Restaurante "LAS PARRILLAS"': [22.2652, -97.8726]}],
 'Carne': [{'Carne Asada El Arbol': [22.283576, -97.886253]},
  {'Carne Asada La Curva': [22.30518146461432, -97.88176138646172]},
  {'Tacos de carne asada El Tío': [22.28

## Folium map

In [58]:
import folium

In [ ]:
top_yf_locs

In [59]:
top_types = df.sort_values(by='score', ascending=False).head().index

mapa=folium.Map(location=[location.latitude, location.longitude], tiles='openstreetmap',zoom_start=12)

five_colors = ['red', 'orange', 'lightgreen', 'green', 'blue']
colors = dict(zip(fs_locs.keys(),five_colors))

folium.Circle([location.latitude, location.longitude], radius=3000, icon=folium.Icon()).add_to(mapa)

for i in top_yf.index:
    folium.Marker(list(top_yf.loc[i][['latitude', 'longitude']]), popup='<b>'+top_yf.loc[i]['name']+'</b>', tooltip=top_yf.loc[i]['tipo'], icon=folium.Icon(color=colors[top_yf.loc[i]['tipo']], icon='info-sign')).add_to(mapa)

for key in top_types:
    for loc in fs_locs[key]:
        name = list(loc.keys())[0]
        folium.Marker(list(loc[name]), popup='<b>'+ name +'</b>', tooltip=key, icon=folium.Icon(color=colors[key], icon='info-sign')).add_to(mapa)

mapa